<a href="https://colab.research.google.com/github/Anis16/-c/blob/main/NAFNet_stereo_Image_Denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAFNet Online Demo on Image Denoising

## Git clone [NAFNet](https://github.com/megvii-research/NAFNet) repo

In [ ]:
!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet

fatal: destination path 'NAFNet' already exists and is not an empty directory.
/content/NAFNet


## Set up the enviroment

In [ ]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

## Download pretrained models

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)

In [ ]:
!gdown https://drive.google.com/uc?id=1TIdQhPtBrZb2wrBdAp9l8NHINLeExOwb -O ./experiments/pretrained_models/
!gdown https://drive.google.com/drive/folders/1zWedm3Q6_6pxTql881DR6xe-v9SPUe70?usp=sharing --folder

## Download Demo Image

In [ ]:
gdown.download('https://drive.google.com/uc?id=1uKwZUgeGfBYLlPKllSuzgGUItlzb40hm', "demo_input/", quiet=False)
gdown.download('https://drive.google.com/uc?id=1ov6UqpIA6GjjJT5SdGeUAJECxka14nGf', "demo_input/", quiet=False)

In [ ]:
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


## Preparation

In [ ]:
import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      
    
      imwrite(sr_img, save_path)
def displaystereo(LR_l, LR_r, SR_l, SR_r):
  h,w = SR_l.shape[:2]
  LR_l = cv2.resize(LR_l, (w,h), interpolation=cv2.INTER_CUBIC)
  LR_r = cv2.resize(LR_r, (w,h), interpolation=cv2.INTER_CUBIC)
  fig = plt.figure(figsize=(w//40, h//40))
  ax1 = fig.add_subplot(2, 2, 1) 
  plt.title('Input image (Left)', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(2, 2, 2)
  plt.title('NAFSSR output (Left)', fontsize=16)
  ax2.axis('off')
  ax1.imshow(LR_l)
  ax2.imshow(SR_l)

  ax3 = fig.add_subplot(2, 2, 3) 
  plt.title('Input image (Right)', fontsize=16)
  ax3.axis('off')
  ax4 = fig.add_subplot(2, 2, 4)
  plt.title('NAFSSR output (Right)', fontsize=16)
  ax4.axis('off')
  ax3.imshow(LR_r)
  ax4.imshow(SR_r)

  plt.subplots_adjust(wspace=0.04, hspace=0.04)

def stereo_image_inference(model, img_l, img_r, save_path):
      img = torch.cat([img_l, img_r], dim=0)
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      img_L = visuals['result'][:,:3]
      img_R = visuals['result'][:,3:]
      img_L, img_R = tensor2img([img_L, img_R])
      img_L = cv2.resize(img_L,(400,400), interpolation = cv2.INTER_AREA)
      img_R = cv2.resize(img_R,(400,400), interpolation = cv2.INTER_AREA)
      imwrite(img_L, save_path.format('L'))
      imwrite(img_R, save_path.format('R'))
def denoising(img_l,img_r):
  save_path='/content/denoised/image1{}.png'
  #output_path='/content/Noisy/image1{}.png'
  img_l = cv2.resize(img_l,(400,400), interpolation = cv2.INTER_AREA)
  img_r = cv2.resize(img_r,(400,400), interpolation = cv2.INTER_AREA)
  img_l= img2tensor(img_l)
  img_r= img2tensor(img_r)
  #stereo_image_inference(NAFSSR, img_l, img_r, output_path)
  #img_input_L = imread(output_path.format('L'))
  #inp_L= img2tensor(img_input_L)
  single_image_inference(NAFNet, img_l, save_path.format('L'))
  #img_input_R = imread(output_path.format('R'))
  #inp_R= img2tensor(img_input_R)
  single_image_inference(NAFNet, img_r, save_path.format('R'))
  return save_path.format('L'),save_path.format('R')
  
  
 

## Create Model

In [ ]:
opt_path1 = 'options/test/SIDD/NAFNet-width64.yml'
opt1 = parse(opt_path1, is_train=False)
opt1['dist'] = False
NAFNet = create_model(opt1)

In [ ]:
opt_path = 'options/test/NAFSSR/NAFSSR-L_4x.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFSSR = create_model(opt)

In [ ]:
def noisy(image,sigma):
      row,col,ch= image.shape
      mean = 0
      image=image/255
      gauss = np.random.normal(mean,sigma,(row,col,ch))/100
      noisy=np.zeros(image.shape)
      for i in range(row):
        for j in range(col):
           noisy[i,j,:]=image[i,j,:]+gauss[i,j,:]
           
           if noisy[i,j,0]>1:
             noisy[i,j,0]=1
           if noisy[i,j,1]>1:
             noisy[i,j,1]=1  
           if noisy[i,j,2]>1:
             noisy[i,j,2]=1  
           if noisy[i,j,0]<0:
             noisy[i,j,0]=0
           if noisy[i,j,1]<0:
             noisy[i,j,1]=0  
           if noisy[i,j,2]<0:
             noisy[i,j,2]=0    
      return noisy*255

Add gaussian Noise to clean image





In [ ]:
import matplotlib.pyplot as plt
import cv2
img1L=cv2.imread('/content/079_L.png')
img1R=cv2.imread('/content/079_R.png')
imglN=noisy(img1L,35)

imgrN=noisy(img1R,35)
#imglN=cv2.cvtColor(imglN,cv2.COLOR_BGR2RGB)

#imgrN=cv2.cvtColor(imgrN,cv2.COLOR_BGR2RGB)
cv2.imwrite('/content/079_{}_noisy_35.png'.format('L'),imglN)
cv2.imwrite('/content/079_{}_noisy_35.png'.format('R'),imgrN)

# Inference and Show results

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
title = "Stereo image denoising "
description = "Stereo image denoising with NAFSSR + denoising NAFNet"
article = "<p style='text-align: center'><a href='https://arxiv.org/abs/2204.04676' target='_blank'>Simple Baselines for Image Restoration</a> | <a href='https://arxiv.org/abs/2204.08714' target='_blank'>NAFSSR: Stereo Image Super-Resolution Using NAFNet</a>  | <a href='https://github.com/megvii-research/NAFNet' target='_blank'> Github Repo</a></p>"


            
iface = gr.Interface(
    denoising, 
    [gr.inputs.Image(type="numpy", label="Input Left"),
     gr.inputs.Image(type="numpy", label="Input Right")
    ], 
    [gr.outputs.Image(type="file", label="Output Left"),
    gr.outputs.Image(type="file", label="Output Right")],
    title=title,
    description=description,
    article=article,
    enable_queue=True,
    
    )
iface.launch(debug=True,enable_queue=True)
      